In [ ]:
import pandas as pd
import numpy as np

In [2]:
event  = pd.read_csv('events_data.csv')
user   = pd.read_csv('device_id.csv')
device  = pd.read_csv('device_brand_model.csv')

In [3]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])


In [4]:
print(event.shape)
print(user.shape)
print(device.shape)

(3252950, 7)
(74645, 5)
(87726, 4)


In [5]:
print('Total no of events :', event['event_id'].nunique())
print('Total no of users  :', user['device_id'].nunique())
print('Total no of devices:', device['device_id'].nunique())
print( 'This means there are {1} users creating {0} events with {2} devices'.format(event['event_id'].nunique(),user['device_id'].nunique(),device['device_id'].nunique()))
print('There are 13109 devices whose user data is not known')

Total no of events : 3252950
Total no of users  : 74645
Total no of devices: 87726
This means there are 74645 users creating 3252950 events with 87726 devices
There are 13109 devices whose user data is not known


In [6]:
event.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   event_id   3252950 non-null  int64  
 1   device_id  3252497 non-null  float64
 2   timestamp  3252950 non-null  object 
 3   longitude  3252527 non-null  float64
 4   latitude   3252527 non-null  float64
 5   city       3252950 non-null  object 
 6   state      3252573 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 173.7+ MB


In [7]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  74645 non-null  int64 
 1   device_id   74645 non-null  int64 
 2   gender      74645 non-null  object
 3   age         74645 non-null  int64 
 4   age_group   74645 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.8+ MB


In [8]:
device.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87726 entries, 0 to 87725
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  87726 non-null  int64 
 1   device_id   87726 non-null  int64 
 2   brand       87726 non-null  object
 3   model       87726 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.7+ MB


In [9]:
missing_data(event)

,Total,Percent
device_id,453,0.013926
latitude,423,0.013004
longitude,423,0.013004
state,377,0.011589
city,0,0.000000
timestamp,0,0.000000
event_id,0,0.000000


In [10]:
missing_data(user)

,Total,Percent
age_group,0,0.0
age,0,0.0
gender,0,0.0
device_id,0,0.0
Unnamed: 0,0,0.0


In [12]:
missing_data(device)

,Total,Percent
model,0,0.0
brand,0,0.0
device_id,0,0.0
Unnamed: 0,0,0.0


In [14]:
event['device_id'] = event['device_id'].astype('object')
user['device_id']  = user['device_id'].astype('object')
device['device_id']= device['device_id'].astype('object')

In [15]:
user.drop('Unnamed: 0', axis = 1, inplace = True)
device.drop('Unnamed: 0', axis = 1, inplace = True)

#### Observation
- There are 74645 users using 87726 devices, 
- That means 13081 are used by same users or has no userid associated with it
- first do an outer joint and then find duplicates

### Event data
- Filling missing states with as per cities
- Filling Latitudes and Longitudes

#### Missing Data
- Latitudes and Longitudes:
- States                  :

In [16]:
event.groupby(['city'])['latitude'].mean()

city
Abohar         30.182626
Achalpur       21.312824
Adilabad       19.735530
Adityapur      22.850015
Adoni          15.673546
                 ...    
Yadgir         16.830949
Yamunanagar    30.185379
Yavatmal       20.462295
Yelahanka      13.160635
Yemmiganur     15.776518
Name: latitude, Length: 933, dtype: float64

In [17]:
event["latitude"].fillna(event.groupby("city")["latitude"].transform("mean"), inplace=True)
event["longitude"].fillna(event.groupby("city")["longitude"].transform("mean"), inplace=True)

In [18]:
state_dict = pd.Series(event.state.values,index=event.city).to_dict()
state_dict

{'Delhi': 'Delhi',
 'Calcutta': 'WestBengal',
 'Chennai': 'TamilNadu',
 'Bokaro': 'Jharkhand',
 'Visakhapatnam': 'AndhraPradesh',
 'Wanparti': 'AndhraPradesh',
 'Mumbai': 'Maharashtra',
 'Pune': 'Maharashtra',
 'Gandhidham': 'Gujarat',
 'Thalassery': 'Kerala',
 'Pusad': 'Maharashtra',
 'Anjangaon': 'Maharashtra',
 'Indore': 'MadhyaPradesh',
 'Paramakkudi': 'TamilNadu',
 'ChikBallapur': 'Karnataka',
 'Gopichettipalaiyam': 'TamilNadu',
 'Nashik': 'Maharashtra',
 'Basmat': 'Maharashtra',
 'Jaipur': 'Rajasthan',
 'Virappanchatram': 'TamilNadu',
 'Sunabeda': 'Orissa',
 'Fazilka': 'Punjab',
 'Aonla': 'UttarPradesh',
 'Adoni': 'AndhraPradesh',
 'Kohima': 'Nagaland',
 'Moga': 'Punjab',
 'Jagadhri': 'Haryana',
 'Kuniyamuthur': 'TamilNadu',
 'Virudhachalam': 'TamilNadu',
 'Thuthukkudi': 'TamilNadu',
 'Sendhwa': 'MadhyaPradesh',
 'Bangalore': 'Karnataka',
 'Hyderabad': 'Telangana',
 'Bodhan': 'AndhraPradesh',
 'Tilhar': 'UttarPradesh',
 'Maheshtala': 'WestBengal',
 'Akola': 'Maharashtra',
 'Bally

In [20]:
def city_mapping(df, dictionary, colsource, coltarget):
    dict_keys = list(dictionary.keys())
    dict_values = list(dictionary.values())
    for x in range(len(dict_keys)):
        df.loc[df[colsource]==dict_keys[x], coltarget] = dict_values[x]
    return(df)

In [21]:
city_mapping(event, state_dict, "city", "state")

,event_id,device_id,timestamp,longitude,latitude,city,state
0,2765368,2.97335e+18,2016-05-07 22:52:05,77.225676,28.730140,Delhi,Delhi
1,2955066,4.73422e+18,2016-05-01 20:44:16,88.388361,22.660325,Calcutta,WestBengal
2,605968,-3.2645e+18,2016-05-02 14:23:04,77.256809,28.757906,Delhi,Delhi
3,448114,5.73137e+18,2016-05-03 13:21:16,80.343613,13.153332,Chennai,TamilNadu
4,665740,3.38888e+17,2016-05-06 03:51:05,85.997745,23.842609,Bokaro,Jharkhand
...,...,...,...,...,...,...,...
3252945,2687452,-1.93703e+18,2016-05-07 23:33:14,73.891597,18.544124,Pune,Maharashtra
3252946,1051580,3.34585e+18,2016-05-03 05:13:30,72.837258,19.018432,Mumbai,Maharashtra
3252947,1316227,-6.40604e+18,2016-05-01 16:03:28,77.235578,28.764065,Delhi,Delhi
3252948,381262,-2.92074e+18,2016-05-05 17:22:36,83.326044,17.765488,Visakhapatnam,AndhraPradesh


In [22]:
event.isnull().sum()

event_id       0
device_id    453
timestamp      0
longitude      0
latitude       0
city           0
state          0
dtype: int64

#### Observation Post Handling Event Data
- Device Id is in exponential format and needs to be changed to numeral
- 453 Device Ids are missing and will be filled when joined with the other two data sets, users and brand.
- Timestamp and event id may be dropped later.

### Device Data
- Changing Chineese names to English
- Drop Model names

In [ ]:
device.head()

In [ ]:
device['brand'].unique()

In [ ]:
device=pd.merge(device,ph_eng,how='left',on='brand')
device.head(3)

In [ ]:
device.drop(['brand','model'], axis=1, inplace = True)

In [ ]:
device.rename(columns={'brand_eng':'brand'}, inplace = True)
device.head()

#### Observation on Device
- Data has device ID and brand names in English

### User Data

#### Age
- Age group has letter and age group
- Modifiy it to remove age group alone
- Age is between 01 to 96
- 5 instance of Age below 10
- 1, 6 is mostlikey a mistaken entry and hence replaced by mode, 26

#### Age Group
- Has Prefix of M and F to denote male and female
- Delete the current group and use a new group from 10 to 90-100
- may drop age or age_group later for modeling or analysis

In [ ]:
user.info()

In [ ]:
user.drop('Unnamed: 0', axis=1, inplace =True)

In [ ]:
user.isnull().sum()

In [ ]:
age=user['age'].unique()
print(age)

In [ ]:
user[user['age'] < 10]

In [ ]:
user.loc[(user.age < 10), 'age'] = 26

In [ ]:
user.isnull().sum()

In [ ]:
user['age_group'].unique()

In [ ]:
user.drop('age_group', axis=1, inplace = True)

In [ ]:
tenure_bins = [0,10,20,30,40,50,60,70,80,100]
tenure_labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-100']
user['age_group'] = pd.cut(user['age'], bins=tenure_bins,labels=tenure_labels,include_lowest=True)
user.tail(5)

### __------------------------Individual Dataset Handling Completed--------------------------__

# __Combining Datasets to One Dataset__

### __Combining Device and User Data__

### Device data to user data
- Maintain all Device ID as users have multiple devices
- Named : device_user

In [ ]:
device_user = pd.merge(device, user, how ='outer')

In [ ]:
device_user.info()

In [ ]:
device_data = pd.merge(device, user, how ='left', on = 'device_id')

In [ ]:
device_data.info()

In [ ]:
duplicate = device_data[device_data['device_id'].duplicated()]

### Observation
- Out of 87726 devices, 
- out of 74645, 28 users have known multiple devices 
- 1309 devices have no users mapped as of now
- There are no duplicate device ids

In [ ]:
event.info(verbose=True,null_counts=True)

In [ ]:
missing_data(event)

In [ ]:
# Placeholder 1001 filled to convert device_id to int to enable mapping
event['device_id'].fillna(value=1001, inplace =True)

In [ ]:
event['device_id']=event['device_id'].astype(int)

In [ ]:
event.head()

In [ ]:
data = pd.merge(event, device_data, how ='right', on ='device_id')

In [ ]:
data.head()

In [ ]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
data.info(verbose=True,null_counts=True)

In [ ]:
missing_data(data)

In [ ]:
print(data['device_id'].duplicated().count())

In [ ]:
state_ap = eud[eud['state']=='AndhraPradesh']
state_hp = eud[eud['state']=='HimachalPradesh']
state_mi = eud[eud['state']=='Mizoram']
state_me = eud[eud['state']=='Meghalaya']
state_po = eud[eud['state']=='Pondicherry']
state_an = eud[eud['state']=='AndamanandNicobarIslands']

In [ ]:
state_data=pd.concat([state_ap, state_hp, state_mi, state_me, state_po, state_an], axis=0)
state_data

In [ ]:
state_data['state'].unique()

In [ ]:
state_data.info()

In [ ]:
duplicate_state_data = state_data[state_data.duplicated()]

In [ ]:
duplicate_state_data

In [ ]:
missing_data(state_data)

In [ ]:
state_data['city'].unique()